# Docker setup

docker run -d -p 8123:8123 -p 9000:9000 --name some-clickhouse-server --ulimit nofile=262144:262144 yandex/clickhouse-server

alias clickhouse-client='docker run -i --rm --link some-clickhouse-server:clickhouse-server yandex/clickhouse-client 
--host clickhouse-server'

echo "show tables" | clickhouse-client 

cat ./analytics_2009-01-01.csv | clickhouse-client --query="INSERT INTO analytics FORMAT CSVWithNames"

### To run client interactively
docker run -it --rm --link some-clickhouse-server:clickhouse-server yandex/clickhouse-client --host clickhouse-server

### Sample query for client
SELECT date, product, mac, instrument, analytic, value FROM analytics WHERE analytic='STAT1' AND product='PROD1' AND mac='MAC1' AND instrument='INST1'

### GUI
http://ui.tabix.io/#!/sql

# Summary results on i5 macbook with 8GB RAM


73.9 million rows with a MergeTree engine and low cardinality strings

Loading from CSV:
approx 4 min.

Great compression:
280MB total table size - with roughly 100Mb mem usage. Versus c. 5Gb in pandas

Queries from notebook for specific analytic, prod, mac, instrument combo return in <2 seconds
Same queries in client and tabix take roughly 4 seconds

## Improvements
Possibly could improve with better primary key definition in the table.
Issues with null/empty string




In [1]:
import pandas as pd
import numpy as np
import itertools
from datetime import datetime as dt
import humanize

In [2]:
def get_test_data(start='20100101', end='20150101', cardinality=2):
    dates = pd.date_range(start=start, end=end, freq='b')
    analytics = ['STAT' + str(i) for i in range(cardinality)]
    instruments = ['INST' + str(i) for i in range(cardinality)] + [None]
    products = ['PROD' + str(i) for i in range(cardinality)] + [None]
    macs = ['MAC' + str(i) for i in range(cardinality)] + [None]
    columns = ["date", "product", "mac", "instrument", "analytic", "value"]
    rows = list(itertools.product(
        dates, products, macs, instruments, analytics))
    data = pd.DataFrame(rows, columns=columns[:-1])
    data['value'] = np.random.random(len(data)) * 10000
    return data

In [47]:
years = pd.date_range(start='20100101', end='20150101', freq='YS')
for y in years:
    start = y - pd.DateOffset(years=1)
    end = y - pd.DateOffset(days=1)
    print((start, end))
    analytics_df = get_test_data(start, end, cardinality=14)
    print(humanize.intword(len(analytics_df)) + ' rows')
    analytics_df.to_csv('./analytics_{:%Y-%m-%d}.csv'.format(start),
                        index=False, date_format='%Y-%m-%d %H:%M:%S')

(Timestamp('2009-01-01 00:00:00'), Timestamp('2009-12-31 00:00:00'))
12.3 million rows
(Timestamp('2010-01-01 00:00:00'), Timestamp('2010-12-31 00:00:00'))
12.3 million rows
(Timestamp('2011-01-01 00:00:00'), Timestamp('2011-12-31 00:00:00'))
12.3 million rows
(Timestamp('2012-01-01 00:00:00'), Timestamp('2012-12-31 00:00:00'))
12.3 million rows
(Timestamp('2013-01-01 00:00:00'), Timestamp('2013-12-31 00:00:00'))
12.3 million rows
(Timestamp('2014-01-01 00:00:00'), Timestamp('2014-12-31 00:00:00'))
12.3 million rows


In [91]:
print(analytics_df.memory_usage().map(humanize.naturalsize))
print(analytics_df.dtypes)

print(analytics_df.info())

Index         80 Bytes
date           98.7 MB
product        98.7 MB
mac            98.7 MB
instrument     98.7 MB
analytic       98.7 MB
value          98.7 MB
dtype: object
date          datetime64[ns]
product               object
mac                   object
instrument            object
analytic              object
value                float64
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12332250 entries, 0 to 12332249
Data columns (total 6 columns):
date          datetime64[ns]
product       object
mac           object
instrument    object
analytic      object
value         float64
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 564.5+ MB
None


In [ ]:
analytics_df.to_csv('./analytics.csv', index=False,
                    date_format='%Y-%m-%d %H:%M:%S')

In [64]:
from clickhouse_driver import Client
client = Client('localhost')

In [77]:
client.execute('SET allow_experimental_low_cardinality_type = 1')
print(client.execute('SHOW TABLES'))
client.execute('DROP TABLE IF EXISTS analytics')
client.execute('''CREATE TABLE analytics (date DateTime,
                                          product LowCardinality(Nullable(String)),
                                          mac LowCardinality(Nullable(String)),
                                          instrument LowCardinality(Nullable(String)),
                                          analytic LowCardinality(Nullable(String)),
                                          value Float32
                                         ) ENGINE MergeTree()
                                         PARTITION BY toYYYYMM(date)
                                         ORDER BY date''')

[]


[]

In [ ]:
# now load with ./load.sh

In [90]:
count_rows = client.execute('SELECT count() FROM analytics')[0][0]
print(humanize.intword(count_rows) + ' rows')

73.9 million rows


In [94]:
def find_analytics(product, mac, instrument, analytic):
    res = client.execute("""SELECT date, product, mac, instrument, analytic, value
                            FROM analytics
                            WHERE analytic=%(analytic)s
                            AND product=%(product)s
                            AND mac=%(mac)s
                            AND instrument=%(instrument)s
                            """,
                         {'product': product,
                          'mac': mac,
                          'instrument': instrument,
                          'analytic': analytic},
                         columnar=False)
    return res

In [96]:
res = find_analytics('PROD1', 'MAC10', 'INST2', 'STAT3')
len(res[0])

6

In [85]:
def pandas_query_at_level_no_index(analytic, level, product=None, mac=None, instrument=None, req_date=None):
    mask = analytics_df.date.notnull()
    filter_dict = dict(product=product, mac=mac,
                       instrument=instrument, analytic=analytic)
    for k, v in filter_dict.items():
        # if k == level:
        #    mask = mask & analytics_df[k].notnull()
        if v is None:
            mask = mask & analytics_df[k].isnull()
        else:
            mask = mask & (analytics_df[k] == v)
    filtered = analytics_df[mask]
    return filtered
    # return filtered.pivot(index='date', columns=level, values='value')

In [86]:
res2 = pandas_query_at_level_no_index(
    'STAT1', 'instrument', product='PROD1', mac='MAC1', instrument='INST1')

In [34]:
len(res2)

1305